In [62]:
import re

from apiclient import discovery
from google.oauth2 import service_account

In [98]:
SCOPES = [
    "https://www.googleapis.com/auth/drive", 
    "https://www.googleapis.com/auth/drive.file", 
    "https://www.googleapis.com/auth/spreadsheets",
]


# The ID of a sample document.
DOCUMENT_ID = '1QBswvLoixphBUoNeSOT4BD9u_jfAGMxCex5-X1Q95gM'

SECRET_FILE = os.path.join(os.getcwd(), 'av-app-365013-07331cd31b9f.json')

#SAMPLE_RANGE_NAME = "List 1!A1:E"
#RANGE = "List 1!C2:I50"
RANGE = "List 1!C3:I50"
PEOPLE_FIELDS = ("name", "code", "section", "section_email", "email", "phone", "city")
SECTIONS_RANGE = "List 1!E3:F50"
SECTIONS_FIELDS = ("name", "email")


In [59]:
def parse_multiple_values(data, separator=";"):
    return data.split(";")


def get_sheet_data(sheet_id, sheet_range):
    credentials = service_account.Credentials.from_service_account_file(SECRET_FILE, scopes=SCOPES)
    service = discovery.build('sheets', 'v4', credentials=credentials)
    
    sheets = service.spreadsheets()
    result = sheets.values().get(spreadsheetId=sheet_id, range=sheet_range).execute()
    
    return result.get("values", [])

In [39]:
get_sheet_data(DOCUMENT_ID, RANGE)

[['Miriam Kudrnová',
  'KUD',
  'vedení',
  'tym.vedeni@absolventskyvelehrad.cz',
  'mk11072018@gmail.com',
  '702 830 714',
  'Brno'],
 ['Milan Janás',
  'JAN',
  'vedení',
  '',
  'milijany31@gmail.com',
  '720 339 904',
  'Praha'],
 ['Jan Kužela',
  'KUZ',
  'technika - ozvučení, záznamy přednášek, atd.',
  'tym.technika@absolventskyvelehrad.cz',
  'kuzela4@seznam.cz',
  '731 870 052',
  'Praha'],
 ['Jiří Sviták',
  'SVI',
  'technika - ozvučení, záznamy přednášek, atd.',
  '',
  'svitak.jiri@gmail.com',
  '737 349 851',
  'Zlín'],
 ['Jarmila Andrlová',
  'AND',
  'finance',
  'tym.finance@absolventskyvelehrad.cz',
  'andrlova.jarka@seznam.cz',
  '774 831 132',
  'Slavkov u Brna '],
 ['Markéta Doleželová',
  'DOL',
  'finance',
  '',
  'dolezelova.market@gmail.com',
  '739 115 599',
  'Brno'],
 ['Martin Ritz',
  'RIT',
  'komunikace, propagace',
  'info@absolventskyvelehrad.cz;tym.propagace@absolventskyvelehrad.cz',
  'fabda4@gmail.com',
  '777 805 786',
  'Ostrava'],
 ['Eliška Sait

In [108]:
def get_section_code(section_name):
    code_regex = re.compile('[^\w]')
    
    code = section_name
    code = code_regex.split(code)[0]
    code = code_regex.sub(" ", code).upper()
    
    return code
    

def get_sections(data, sheet_range=SECTIONS_RANGE):
    for row in data:
        # Extend row result with trailing None values
        row = row + ([None] * (len(SECTIONS_FIELDS) - len(row)))
        row_data = dict(zip(SECTIONS_FIELDS, row))

        if not row_data.get("email"):
            continue

        row_data["email"] = parse_multiple_values(row_data["email"])
        row_data["code"] = get_section_code(row_data["name"])

        yield row_data
    
    

In [109]:
list(get_sections(get_sheet_data(DOCUMENT_ID, SECTIONS_RANGE)))

[{'name': 'vedení',
  'email': ['tym.vedeni@absolventskyvelehrad.cz'],
  'code': 'VEDENÍ'},
 {'name': 'technika - ozvučení, záznamy přednášek, atd.',
  'email': ['tym.technika@absolventskyvelehrad.cz'],
  'code': 'TECHNIKA'},
 {'name': 'finance',
  'email': ['tym.finance@absolventskyvelehrad.cz'],
  'code': 'FINANCE'},
 {'name': 'komunikace, propagace',
  'email': ['info@absolventskyvelehrad.cz',
   'tym.propagace@absolventskyvelehrad.cz'],
  'code': 'KOMUNIKACE'},
 {'name': 'realizace',
  'email': ['tym.realizace@absolventskyvelehrad.cz'],
  'code': 'REALIZACE'},
 {'name': 'dobrovolníci',
  'email': ['tym.dobrovolnici@absolventskyvelehrad.cz'],
  'code': 'DOBROVOLNÍCI'},
 {'name': 'duchovní',
  'email': ['tym.duchovni@absolventskyvelehrad.cz'],
  'code': 'DUCHOVNÍ'},
 {'name': 'přednášky',
  'email': ['tym.prednasky@absolventskyvelehrad.cz'],
  'code': 'PŘEDNÁŠKY'},
 {'name': 'volnočasovky',
  'email': ['tym.volnocasovky@absolventskyvelehrad.cz'],
  'code': 'VOLNOČASOVKY'},
 {'name': 

In [118]:
def get_people(data):
    for row in data:
        row_data = dict(zip(PEOPLE_FIELDS, row))

        if not row_data.get('code'):
            continue
            
        row_data.pop("section_email", None)
        row_data["section"] = get_section_code(row_data["section"])
        row_data["email"] = parse_multiple_values(row_data["email"])
        row_data["phone"] = parse_multiple_values(row_data["phone"])
            
        yield row_data

In [119]:
list(get_people(get_sheet_data(DOCUMENT_ID, RANGE)))

[{'name': 'Miriam Kudrnová',
  'code': 'KUD',
  'section': 'VEDENÍ',
  'email': ['mk11072018@gmail.com'],
  'phone': ['702 830 714'],
  'city': 'Brno'},
 {'name': 'Milan Janás',
  'code': 'JAN',
  'section': 'VEDENÍ',
  'email': ['milijany31@gmail.com'],
  'phone': ['720 339 904'],
  'city': 'Praha'},
 {'name': 'Jan Kužela',
  'code': 'KUZ',
  'section': 'TECHNIKA',
  'email': ['kuzela4@seznam.cz'],
  'phone': ['731 870 052'],
  'city': 'Praha'},
 {'name': 'Jiří Sviták',
  'code': 'SVI',
  'section': 'TECHNIKA',
  'email': ['svitak.jiri@gmail.com'],
  'phone': ['737 349 851'],
  'city': 'Zlín'},
 {'name': 'Jarmila Andrlová',
  'code': 'AND',
  'section': 'FINANCE',
  'email': ['andrlova.jarka@seznam.cz'],
  'phone': ['774 831 132'],
  'city': 'Slavkov u Brna '},
 {'name': 'Markéta Doleželová',
  'code': 'DOL',
  'section': 'FINANCE',
  'email': ['dolezelova.market@gmail.com'],
  'phone': ['739 115 599'],
  'city': 'Brno'},
 {'name': 'Martin Ritz',
  'code': 'RIT',
  'section': 'KOMUNIK

In [1]:
from imports.sheets import TeamImportService

In [2]:
avs = TeamImportService()

In [5]:
[p["name"] for p in avs.list_sections()]

['vedení',
 'technika - ozvučení, záznamy přednášek, atd.',
 'finance',
 'komunikace, propagace',
 'realizace',
 'dobrovolníci',
 'duchovní',
 'přednášky',
 'volnočasovky',
 'IT, merch, grafika',
 'čajovna/vinárna',
 'schola']